In [1]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import os
import glob
import pydicom
import matplotlib.pyplot as plt
from time import time_ns
import sys
sys.path.append(os.getcwd())
sys.path.append(os.getcwd() + "/rsna-2024/")
from rsna_dataloader import create_datasets_and_loaders, clean_training_data, retrieve_training_data
import torch

In [2]:
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.Lambda(lambda x: (x * 255).astype(np.uint8)),  # Convert back to uint8 for PIL
    transforms.ToPILImage(),
    transforms.Resize((512, 512)),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
])


In [13]:
label_map = {'normal_mild': 0, 'moderate': 1, 'severe': 2}
series_desc = ('Sagittal T1', 'Axial T2', 'Sagittal T2/STIR')
data = retrieve_training_data("C:/Users/Victor/Documents/python-doodles/data/rsna-2024-lumbar-spine-degenerative-classification/")
data = clean_training_data(data, "C:/Users/Victor/Documents/python-doodles/data/rsna-2024-lumbar-spine-degenerative-classification/")
data = data.dropna()

loaders = {e : create_datasets_and_loaders(data, e, transform, batch_size=4) for e in series_desc}
loaders 

{'Sagittal T1': (<torch.utils.data.dataloader.DataLoader at 0x284b7a42890>,
  15755,
  3939),
 'Axial T2': (<torch.utils.data.dataloader.DataLoader at 0x284b7b49390>,
  15372,
  3843),
 'Sagittal T2/STIR': (<torch.utils.data.dataloader.DataLoader at 0x286eda87410>,
  7798,
  1950)}

In [ ]:
# import importlib
# import unet
# import train_unet
# importlib.reload(unet)
# importlib.reload(train_unet)
# from train_unet import train
# from unet import UNet
# models = {e : UNet(1, 3) for e in loaders}

In [11]:
def trainable_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [7]:
# train(loaders[e][0], loaders[e][1], label_map)

In [5]:
from fastmri.models.unet import Unet

In [6]:
model = Unet(num_pool_layers=4, drop_prob=0.05, in_chans=1, out_chans=1)
print(trainable_params(model))
#model.load_weights('/Users/victorsahin/PycharmProjects/pythonProject/models/unet_model_weights.h5')

7756097


In [7]:
import torch.nn as nn

class ConvToFC(nn.Module):
    def __init__(self, in_channels, in_dims, out_dims):
        super(ConvToFC, self).__init__()
        self.conv = nn.Conv2d(in_channels, 1, kernel_size=(in_dims, 1))
        self.relu = nn.ReLU(inplace=True)
        self.fc = nn.Linear(in_dims, out_dims)
    def forward(self, x):
        return self.fc(self.relu(self.conv(x)))

class UNetClassifier(Unet):
    def __init__(self):
        super().__init__(1, 64)
        self.conv_to_fc = ConvToFC(64, 512, 3)
    
    def forward(self, x):
        return self.conv_to_fc(super().forward(x))

In [8]:
models = {e: UNetClassifier().to("cuda") for e in series_desc}
models

{'Sagittal T1': UNetClassifier(
   (down_sample_layers): ModuleList(
     (0): ConvBlock(
       (layers): Sequential(
         (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
         (1): InstanceNorm2d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
         (2): LeakyReLU(negative_slope=0.2, inplace=True)
         (3): Dropout2d(p=0.0, inplace=False)
         (4): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
         (5): InstanceNorm2d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
         (6): LeakyReLU(negative_slope=0.2, inplace=True)
         (7): Dropout2d(p=0.0, inplace=False)
       )
     )
     (1): ConvBlock(
       (layers): Sequential(
         (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
         (1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
         (2): LeakyReLU(negative_sl

In [ ]:
import importlib
import train_unet
importlib.reload(train_unet)
from train_unet import train
res = {e: train(loaders[e][0], loaders[e][1], label_map, model=models[e], model_desc="unet_" + e.replace(" ", "").replace("/", ""), epoch_count=12) for e in series_desc}
res

  0%|          | 2/3939 [00:01<41:56,  1.56it/s]

In [67]:
model.eval()

UNetClassifier(
  (down_sample_layers): ModuleList(
    (0): ConvBlock(
      (layers): Sequential(
        (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): InstanceNorm2d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (2): LeakyReLU(negative_slope=0.2, inplace=True)
        (3): Dropout2d(p=0.0, inplace=False)
        (4): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (5): InstanceNorm2d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (6): LeakyReLU(negative_slope=0.2, inplace=True)
        (7): Dropout2d(p=0.0, inplace=False)
      )
    )
    (1): ConvBlock(
      (layers): Sequential(
        (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (2): LeakyReLU(negative_slope=0.2, inplace=True)
        (3)

In [64]:
# torch.backends.quantized.engine = 'qnnpack'
# model_quantized = torch.ao.quantization.quantize_dynamic(
#     model,  # the original model
#     {torch.nn.ConvTranspose2d, torch.nn.Linear},  # a set of layers to dynamically quantize
#     dtype=torch.qint8)

In [65]:
# model_quantized = train(loaders["Sagittal T1"][0], loaders["Sagittal T1"][1], label_map, model=model_quantized)

  0%|          | 0/1973 [00:10<?, ?it/s]


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn